In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests

In [174]:
pokemon = pd.DataFrame(columns = ["number", "name", "type", "ability", "hidden_ability", 
                                 "gender_ratio", "hatch_cycle","egg_group", "Base_Experience_Yield", "height", "weight",
                                 "leveling_rate", "color", "base_friendship", "category"], index = range(1, 1026))
pokemon

,number,name,type,ability,hidden_ability,gender_ratio,hatch_cycle,egg_group,Base_Experience_Yield,height,weight,leveling_rate,color,base_friendship,category
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
pokemon = pd.read_csv("/home/user/Desktop/Data_analysis_projects/Pokemon/data.csv", index_col = 0)


In [43]:
# Creating a function for scraping pokemon data

def scrap_pokemon_details(pokemon_name):
    """
    scrape details of pokemon from bulbapedia

    Args :
        pokemon_name(str) name of the pokemon, eg : Greninja 

    Returns:
        A dictionary will all the details 
    """
    
    try: 
        # Give URL 
        url = f"https://bulbapedia.bulbagarden.net/wiki/{pokemon_name}_(Pok%C3%A9mon)"
        
        # fetching the webpage 
        response = requests.get(url)
        response.raise_for_status()
        text = response.text
        
        # Creating a BeautifulSoup instance 
        soup = BeautifulSoup(text, "lxml")
        
        # Overall data 
        data = soup.find_all("td", class_ = "roundy")
        
        if len(data) < 40 :
            print(f"Don't have enought data for {pokemon_data}")
            return None
        
        # name, number, category
        basic = data[0]
        name = basic.find(["big", "b"]).text.strip()
        category = basic.find(["span"]).text.strip()
        number = soup.find(["th","span", "big"], class_ = "roundy").text.replace("\n", "").strip()
        
        # Type 
        type_data = data[1]
        type_data_2 =  type_data.find_all(["span", "a"])
        if type_data_2[4].text.strip() != "Unknown":
            type_ = f"{type_data_2[2].text.strip()} / {type_data_2[4].text.strip()}"
        else:
            type_ = type_data_2[2].text.strip()
        
        # ability and hidden ability
        ability_data = data[2]
        ability_data2 = ability_data.find_all(["a", "span"])
        clear_ability = []
        junk_text = ["Cacophony", "Abilities", "Gen IV+", "Battle Bond", "Battle Bond Greninja", "Gen VI+", "Gen VII+", "Ability", "Gen VIII+"]

        for i in ability_data2:
            text = i.text.strip()
            if text and text not in junk_text:
                clear_ability.append(text)
        
        if len(clear_ability) == 6:
            ability = f"{clear_ability[0]} / {clear_ability[2]}"
            hidden_ability = clear_ability[4]
        elif len(clear_ability) == 4:
            ability = clear_ability[0]
            hidden_ability = clear_ability[2]
        elif len(clear_ability) == 2:
            ability = clear_ability[0]
            hidden_ability = None
        else:
            ability = None
            hidden_ability = None
            
        # gender ratio
        gender_data = data[3]
        gender_data_2 = gender_data.find_all(["a"])
        gender_ratio = gender_data_2[1].text.strip()
        
        # Catch rate 
        catch_rate = data[4].find("small").text.strip()
        
        # egg group and hatch cycle 
        egg_group_data = data[5]
        egg_group = egg_group_data.find("table", class_ = "roundy").text.strip()
        hatch_cycle_data = data[5].find_all("table", class_ = "roundy")
        hatch_cycle = hatch_cycle_data[1].text.strip().replace("\xa0", "").replace("Egg not obtainable", "")
        
        # Height and Weight 
        height_data = data[6].find_all("tr")
        height_data_2 = height_data[0].find_all("td")
        height = height_data_2[1].text.replace("\n", "").strip()
        weight_data = data[7].find_all("tr")
        weight_data_2 = weight_data[0].find_all("td")
        weight = weight_data_2[1].text.replace("\n", "").strip()
        
        # Base Experience Yield 
        bey_data = data[9].find("tr")
        bey_data_2 = bey_data.find_all("td")
        bey = bey_data_2[2].text.replace("V+", "").replace("\n", "").strip()
        
        # Other details 
        leveling_rate = data[10].tr.text.strip()
        color = data[38].td.text.strip().replace("Other forms may have other colors.", "")
        base_friendship = data[39].tr.text.strip()
        
        return {"number" : number, "name" : name, "type" : type_, "ability" : ability, "hidden_ability" : hidden_ability, 
                 "gender_ratio" : gender_ratio, "hatch_cycle" : hatch_cycle, "egg_group" : egg_group, "Base_Experience_Yield" : bey, "leveling_rate" : leveling_rate,
                 "color" : color, "base_friendship" : base_friendship, "height" : height, "weight" : weight, "category" : category}
            
    except requests.RequestException as e:
        print(f"Failed to fetch data for {pokemon_name}: {e}")
        return None
    except (AttributeError, IndexError) as e:
        print(f"Failed to parse data for {pokemon_name}: {e}")
        return None

            

In [4]:
pokemon.loc[0] = None

In [242]:
# Bulbasaur evolution line
bulbasaur_data = scrap_pokemon_details("Bulbasaur")
pokemon.loc[1] = bulbasaur_data
ivysaur_data = scrap_pokemon_details("Ivysaur")
pokemon.loc[2] = ivysaur_data
venusaur_data = scrap_pokemon_details("Venusaur")
pokemon.loc[3] = venusaur_data

In [180]:
# Charmander line 
charmander_data = scrap_pokemon_details("Charmander")
pokemon.loc[4] = charmander_data
charmeleon_data = scrap_pokemon_details("Charmeleon")
pokemon.loc[5] = charmeleon_data
charizard_data = scrap_pokemon_details("Charizard")
charizard_data["ability"] = "Blaze"
charizard_data["hidden_ability"] = "Solar Power"
pokemon.loc[6] = charizard_data

In [181]:
# Squirtle line 
squirtle_data = scrap_pokemon_details("Squirtle")
pokemon.loc[7] = squirtle_data
wartortle_data = scrap_pokemon_details("Wartortle")
pokemon.loc[8] = wartortle_data
blastoise_data = scrap_pokemon_details("Blastoise")
pokemon.loc[9] = blastoise_data

In [182]:
# Caterpie line 
caterpie_data = scrap_pokemon_details("Caterpie")
pokemon.loc[10] = caterpie_data
metapod_data = scrap_pokemon_details("Metapod")
pokemon.loc[11] = metapod_data
butterfree_data = scrap_pokemon_details("Butterfree")
pokemon.loc[12] = butterfree_data

In [183]:
# Weedle line 
weedle_data = scrap_pokemon_details("Weedle")
pokemon.loc[13] = weedle_data
kakuna_data = scrap_pokemon_details("Kakuna")
pokemon.loc[14] = kakuna_data
beedrill_data = scrap_pokemon_details("Beedrill")
pokemon.loc[15] = beedrill_data

In [184]:
#pidgey line
pidgey_data = scrap_pokemon_details("Pidgey")
pokemon.loc[16] = pidgey_data
pidgeotto_data = scrap_pokemon_details("Pidgeotto")
pokemon.loc[17] = pidgeotto_data
pidgeot_data = scrap_pokemon_details("Pidgeot")
pidgeot_data["ability"] = "Keen Eye / Tangled Feet"
pidgeot_data["hidden_ability"] = "Big Pecks"
pokemon.loc[18] = pidgeot_data

In [185]:
# Rattata line
rattata_data = scrap_pokemon_details("Rattata")
rattata_data["ability"] = "Run Away / Guts"
rattata_data["hidden_ability"] = "Hustle"
pokemon.loc[19] = rattata_data
raticate_data = scrap_pokemon_details("Raticate")
raticate_data["ability"] = "Run Away / Guts"
raticate_data["hidden_ability"] = "Hustle"
pokemon.loc[20] = raticate_data

In [186]:
#Spearow line
spearow_data = scrap_pokemon_details("Spearow")
pokemon.loc[21] = spearow_data
fearow_data = scrap_pokemon_details("Fearow")
pokemon.loc[22] = fearow_data

In [187]:
# Ekans line
ekans_data = scrap_pokemon_details("Ekans")
pokemon.loc[23] = ekans_data
arbok_data = scrap_pokemon_details("Arbok")
pokemon.loc[24] = arbok_data

In [188]:
#Pikachu line
pikachu_data = scrap_pokemon_details("Pikachu")
pokemon.loc[25] = pikachu_data
raichu_data = scrap_pokemon_details("Raichu")
raichu_data["ability"] = "Static"
pokemon.loc[26] = raichu_data


In [189]:
#Sandshrew line
sandshrew_data = scrap_pokemon_details("Sandshrew")
sandshrew_data["ability"] = "Sand Veil"
sandshrew_data["hidden_ability"] = "Sand Rush"
pokemon.loc[27] = sandshrew_data
sandslash_data = scrap_pokemon_details("Sandslash")
sandslash_data["ability"] = "Sand Veil"
sandslash_data["hidden_ability"] = "Sand Rush"
pokemon.loc[28] = sandslash_data

In [190]:
# nidoran female line
nidoranf_data = scrap_pokemon_details("Nidoran♀")
pokemon.loc[29] = nidoranf_data
nidorina_data = scrap_pokemon_details("Nidorina")
pokemon.loc[30] = nidorina_data
nidoqueen_data = scrap_pokemon_details("Nidoqueen")
pokemon.loc[31] = nidoqueen_data

In [191]:
# Nidoran male line
nidoranm_data = scrap_pokemon_details("Nidoran♂")
pokemon.loc[32] = nidoranm_data
nidorino_data = scrap_pokemon_details("Nidorino")
pokemon.loc[33] = nidorino_data
nidoking_data = scrap_pokemon_details("Nidoking")
pokemon.loc[34] = nidoking_data

In [192]:
#Clefairy line 
clefairy_data = scrap_pokemon_details("Clefairy")
pokemon.loc[35] = clefairy_data
clefable_data = scrap_pokemon_details("Clefable")
pokemon.loc[36] = clefable_data

In [193]:
# Vulpix line
vulpix_data = scrap_pokemon_details("Vulpix")
vulpix_data["ability"] = "Flash Fire"
vulpix_data["hidden_ability"] = "Drought"
pokemon.loc[37] = vulpix_data
ninetails_data = scrap_pokemon_details("Ninetales")
ninetails_data["ability"] = "Flash Fire"
ninetails_data["hidden_ability"] = "Drought"
pokemon.loc[38] = ninetails_data

In [194]:
# Jigglypuff line
jigglypuff_data = scrap_pokemon_details("Jigglypuff")
pokemon.loc[39] = jigglypuff_data
wigglytuff_data = scrap_pokemon_details("Wigglytuff")
pokemon.loc[40] = wigglytuff_data

In [195]:
#Zubat line
zubat_data = scrap_pokemon_details("Zubat")
pokemon.loc[41] = zubat_data
golbat_data = scrap_pokemon_details("Golbat")
pokemon.loc[42] = golbat_data

In [196]:
#Oddish line
oddish_data = scrap_pokemon_details("Oddish")
pokemon.loc[43] = oddish_data
gloom_data = scrap_pokemon_details("Gloom")
pokemon.loc[44] = gloom_data
vileplume_data = scrap_pokemon_details("Vileplume")
pokemon.loc[45] = vileplume_data

In [197]:
# paras line
paras_data = scrap_pokemon_details("Paras")
pokemon.loc[46] = paras_data
parasect_data = scrap_pokemon_details("Parasect")
pokemon.loc[47] = parasect_data

In [198]:
# Venonat line
venonat_data = scrap_pokemon_details("Venonat")
pokemon.loc[48] = venonat_data
venomoth_data = scrap_pokemon_details("Venomoth")
pokemon.loc[49] = venomoth_data

In [199]:
# diglett line
diglett_data = scrap_pokemon_details("Diglett")
diglett_data["ability"] = "Sand Veil / Arena Trap"
diglett_data["hidden_ability"] = "Sand Force"
pokemon.loc[50] = diglett_data
dugtrio_data = scrap_pokemon_details("Dugtrio")
dugtrio_data["ability"] = "Sand Veil / Arena Trap"
dugtrio_data["hidden_ability"] = "Sand Force"
pokemon.loc[51] = dugtrio_data

In [200]:
# meowth line
meowth_data = scrap_pokemon_details("Meowth")
meowth_data["ability"] = "Pickup", "Technician"
meowth_data["hidden_ability"] = "Unnerve"
pokemon.loc[52] = meowth_data
persian_data = scrap_pokemon_details("Persian")
persian_data["ability"] = "Limber", "Technician"
persian_data["hidden_ability"] = "Unnerve"
pokemon.loc[53] = persian_data 

In [201]:
# Psyduck line
psyduck_data = scrap_pokemon_details("Psyduck")
pokemon.loc[54] = psyduck_data
golduck_data = scrap_pokemon_details("Golduck")
pokemon.loc[55] = golduck_data

In [202]:
# Mankey line 
mankey_data = scrap_pokemon_details("Mankey")
pokemon.loc[56] = mankey_data
primeape_data = scrap_pokemon_details("Primeape")
pokemon.loc[57] = primeape_data

In [32]:
# Growlithe line
growlithe_data = scrap_pokemon_details("Growlithe")
growlithe_data["ability"] = "Intimidate / Flash Fire"
growlithe_data["hidden_ability"] = "Justified"
growlithe_data["category"] = "Puppy Pokemon"
pokemon.loc[58] = growlithe_data
arcanine_data = scrap_pokemon_details("Arcanine")
arcanine_data["ability"] = "Intimidate / Flash Fire"
arcanine_data["hidden_ability"] = "Justified"
pokemon.loc[59] = arcanine_data

In [204]:
# poliwag line
poliwag_data = scrap_pokemon_details("Poliwag")
pokemon.loc[60] = poliwag_data
poliwhirl_data = scrap_pokemon_details("Poliwhirl")
pokemon.loc[61] = poliwhirl_data
poliwrath_data = scrap_pokemon_details("Poliwrath")
pokemon.loc[62] = poliwrath_data

In [205]:
# Abra line 
abra_data = scrap_pokemon_details("Abra")
pokemon.loc[63] = abra_data
kadabra_data = scrap_pokemon_details("Kadabra")
pokemon.loc[64] = kadabra_data
alakazam_data = scrap_pokemon_details("Alakazam")
alakazam_data["ability"] = "Synchronize / Inner Focus"
alakazam_data["hidden_ability"] = "Magic Guard"
pokemon.loc[65] = alakazam_data

In [206]:
# Machop line
machop_data = scrap_pokemon_details("Machop")
pokemon.loc[66] = machop_data
machoke_data = scrap_pokemon_details("Machoke")
pokemon.loc[67] = machoke_data
machamp_data = scrap_pokemon_details("Machamp")
pokemon.loc[68] = machamp_data

In [207]:
# bellsprout line 
bellsprout_data = scrap_pokemon_details("Bellsprout")
pokemon.loc[69] = bellsprout_data
weepinbell_data = scrap_pokemon_details("Weepinbell")
pokemon.loc[70] = weepinbell_data
victreebel_data = scrap_pokemon_details("Victreebel")
pokemon.loc[71] = victreebel_data

In [208]:
# tentacool line
tentacool_data = scrap_pokemon_details("Tentacool")
pokemon.loc[72] = tentacool_data
tentacruel_data = scrap_pokemon_details("Tentacruel")
pokemon.loc[73] = tentacruel_data

In [209]:
# geodude line 
geodude_data = scrap_pokemon_details("Geodude")
geodude_data["ability"] = "Rock Head / Sturdy"
geodude_data["hidden_ability"] = "Sand Veil"
pokemon.loc[74] = geodude_data
graveler_data = scrap_pokemon_details("Graveler")
graveler_data["ability"] = "Rock Head / Sturdy"
graveler_data["hidden_ability"] = "Sand Veil"
pokemon.loc[75] = graveler_data
golem_data = scrap_pokemon_details("Golem")
golem_data["ability"] = "Rock Head / Sturdy"
golem_data["hidden_ability"] = "Sand Veil"
pokemon.loc[76] = golem_data

In [31]:
# ponyta line 
ponyta_data = scrap_pokemon_details("Ponyta")
ponyta_data["ability"] = "Run Away", "Flash Fire"
ponyta_data["hidden_ability"] = "Flame Body"
ponyta_data["category"] = "Fire Horse Pokémon"
pokemon.loc[77] = ponyta_data
rapidash_data = scrap_pokemon_details("Rapidash")
rapidash_data["ability"] = "Run Away", "Flash Fire"
rapidash_data["hidden_ability"] = "Flame Body"
rapidash_data["category"] = "Fire Horse Pokémon"
pokemon.loc[78] = rapidash_data

In [211]:
# Slowpoke line 
slowpoke_data = scrap_pokemon_details("Slowpoke")
slowpoke_data["ability"] = "Oblivious / Own Tempo"
slowpoke_data["hidden_ability"] = "Regenerator"
pokemon.loc[79] = slowpoke_data
slowbro_data = scrap_pokemon_details("Slowbro")
slowbro_data["ability"] = "Oblivious / Own Tempo"
slowbro_data["hidden_ability"] = "Regenerator"
pokemon.loc[80] = slowbro_data

In [212]:
# magnemite line
magnemite_data = scrap_pokemon_details("Magnemite")
pokemon.loc[81] = magnemite_data
magneton_data = scrap_pokemon_details("Magneton")
pokemon.loc[82] = magneton_data

In [213]:
# Farfetch'd
farfetch_data = scrap_pokemon_details("Farfetch'd")
farfetch_data["ability"] = "Keen Eye / Inner Focus"
farfetch_data["hidden_ability"] = "Defiant"
pokemon.loc[83] = farfetch_data

In [214]:
# doduo line
doduo_data = scrap_pokemon_details("Doduo")
pokemon.loc[84] = doduo_data
dodrio_data = scrap_pokemon_details("Dodrio")
pokemon.loc[85] = dodrio_data

In [215]:
# Seel line 
seel_data = scrap_pokemon_details("Seel")
pokemon.loc[86] = seel_data
dewgong_data = scrap_pokemon_details("Dewgong")
pokemon.loc[87] = dewgong_data

In [216]:
# grimer line 
grimer_data = scrap_pokemon_details("Grimer")
grimer_data["ability"] = "Stench / Sticky Hold"
grimer_data["hidden_ability"] = "Poison Touch"
pokemon.loc[88] = grimer_data
muk_data = scrap_pokemon_details("Muk")
muk_data["ability"] = "Stench / Sticky Hold"
muk_data["hidden_ability"] = "Poison Touch"
pokemon.loc[89] = muk_data

In [217]:
# Shellder line 
shellder_data = scrap_pokemon_details("Shellder")
pokemon.loc[90] = shellder_data
cloyster_data = scrap_pokemon_details("Cloyster")
pokemon.loc[91] = cloyster_data

In [218]:
# Gastly line
gastly_data = scrap_pokemon_details("Gastly")
pokemon.loc[92] = gastly_data
haunter_data = scrap_pokemon_details("Haunter")
pokemon.loc[93] = haunter_data
gengar_data = scrap_pokemon_details("Gengar")
gengar_data["ability"] = "Cursed Body"
pokemon.loc[94] = gengar_data

In [219]:
# onix line 
onix_data = scrap_pokemon_details("Onix")
pokemon.loc[95] = onix_data

In [220]:
# drowzee line 
drowzee_data = scrap_pokemon_details("Drowzee")
pokemon.loc[96] = drowzee_data
hypno_data = scrap_pokemon_details("Hypno")
pokemon.loc[97] = hypno_data

In [221]:
# Krabby line 
krabby_data = scrap_pokemon_details("Krabby")
pokemon.loc[98] = krabby_data
kingler_data = scrap_pokemon_details("Kingler")
pokemon.loc[99] = kingler_data

In [30]:
# voltorb line 
voltorb_data = scrap_pokemon_details("Voltorb")
voltorb_data["category"] = "Ball Pokemon"
pokemon.loc[100] = voltorb_data
electrode_data = scrap_pokemon_details("Electrode")
electrode_data["category"] = "Ball Pokemon"
pokemon.loc[101] = electrode_data

In [223]:
# Exeggcute line 
exeggcute_data = scrap_pokemon_details("Exeggcute")
pokemon.loc[102] = exeggcute_data
exeggutor_data = scrap_pokemon_details("Exeggutor")
pokemon.loc[103] = exeggutor_data

In [224]:
# cubone line 
cubone_data = scrap_pokemon_details("Cubone")
pokemon.loc[104] = cubone_data
marowak_data = scrap_pokemon_details("Marowak")
marowak_data["ability"] = "Rock Head / Lightning Rod"
marowak_data["hidden_ability"] = "Battle Armor"
pokemon.loc[105] = marowak_data

In [225]:
# hitmonlee line  and lickitung
hitmonlee_data = scrap_pokemon_details("Hitmonlee")
pokemon.loc[106] = hitmonlee_data
hitmonchan_data = scrap_pokemon_details("Hitmonchan")
pokemon.loc[107] = hitmonchan_data
lickitung_data = scrap_pokemon_details("Lickitung")
pokemon.loc[108] = lickitung_data

In [226]:
# koffing
koffing_data = scrap_pokemon_details("Koffing")
koffing_data["ability"] = "Levitate / Neutralizing Gas"
koffing_data["hidden_ability"] = "Stench"
pokemon.loc[109] = koffing_data
weezing_data = scrap_pokemon_details("Weezing")
weezing_data["ability"] = "Levitate / Neutralizing Gas"
weezing_data["hidden_ability"] = "Stench"
pokemon.loc[110] = weezing_data

In [227]:
# Rhyhorn line
rhyhorn_data = scrap_pokemon_details("Rhyhorn")
pokemon.loc[111] = rhyhorn_data
rhydon_data = scrap_pokemon_details("Rhydon")
pokemon.loc[112] = rhydon_data

In [228]:
# Chansey line and tangela line
chansey_data = scrap_pokemon_details("Chansey")
pokemon.loc[113] = chansey_data
tangela_data = scrap_pokemon_details("Tangela")
pokemon.loc[114] = tangela_data

In [229]:
#Kangaskhan line
kangaskhan_data = scrap_pokemon_details("Kangaskhan")
kangaskhan_data["ability"] = "Early Bird / Scrappy"
kangaskhan_data["hidden_ability"] = "Inner Focus"
pokemon.loc[115] = kangaskhan_data

In [230]:
# horsea line 
horsea_data = scrap_pokemon_details("Horsea")
pokemon.loc[116] = horsea_data
seadra_data = scrap_pokemon_details("Seadra")
pokemon.loc[117] = seadra_data

In [231]:
# goldeen and staryu line
goldeen_data = scrap_pokemon_details("Goldeen")
pokemon.loc[118] = goldeen_data
seaking_data = scrap_pokemon_details("Seaking")
pokemon.loc[119] = seaking_data
staryu_data = scrap_pokemon_details("Staryu")
pokemon.loc[120] = staryu_data
starmie_data = scrap_pokemon_details("Starmie")
pokemon.loc[121] = starmie_data

In [29]:
# Mr mime line 
mrmime_data = scrap_pokemon_details("Mr._Mime")
mrmime_data["ability"] = "Soundproof / Filter"
mrmime_data["hidden_ability"] = "Technician"
mrmime_data["category"] = "Barrier Pokemon"
pokemon.loc[122] = mrmime_data

In [233]:
# Scyther and jynx line 
scyther_data = scrap_pokemon_details("Scyther")
pokemon.loc[123] = scyther_data
jynx_data = scrap_pokemon_details("Jynx")
pokemon.loc[124] = jynx_data

In [234]:
# Electabuzz and Magmar and pinsir
electabuzz_data = scrap_pokemon_details("Electabuzz")
pokemon.loc[125] = electabuzz_data
magmar_data = scrap_pokemon_details("Magmar")
pokemon.loc[126] = magmar_data
pinsir_data = scrap_pokemon_details("Pinsir")
pinsir_data["ability"] = "Hyper Cutter / Mold Breaker"
pinsir_data["hidden_ability"] = "Moxie"
pokemon.loc[127] = pinsir_data

In [235]:
# Tauros and magicarp line
tauros_data = scrap_pokemon_details("Tauros")
tauros_data["ability"] = "Intimidate / Anger Point"
tauros_data["hidden_ability"] = "Sheer Force"
pokemon.loc[128] = tauros_data
magikarp_data = scrap_pokemon_details("Magikarp")
pokemon.loc[129] = magikarp_data
gyarados_data = scrap_pokemon_details("Gyarados")
gyarados_data["ability"] = "Intimidate"
gyarados_data["hidden_ability"] = "Moxie"
pokemon.loc[130] = gyarados_data

In [236]:
# Lapras, ditto
lapras_data = scrap_pokemon_details("Lapras")
pokemon.loc[131] = lapras_data
ditto_data = scrap_pokemon_details("Ditto")
pokemon.loc[132] = ditto_data

In [237]:
# Eevee line
eevee_data = scrap_pokemon_details("Eevee")
pokemon.loc[133] = eevee_data
vaporeon_data = scrap_pokemon_details("Vaporeon")
pokemon.loc[134] = vaporeon_data
jolteon_data = scrap_pokemon_details("Jolteon")
pokemon.loc[135] = jolteon_data
flareon_data = scrap_pokemon_details("Flareon")
pokemon.loc[136] = flareon_data

In [238]:
# porygon and omanyte and kabuto line 
porygon_data = scrap_pokemon_details("Porygon")
pokemon.loc[137] = porygon_data
omanyte_data = scrap_pokemon_details("Omanyte")
pokemon.loc[138] = omanyte_data
omastar_data = scrap_pokemon_details("Omastar")
pokemon.loc[139] = omastar_data
kabuto_data = scrap_pokemon_details("Kabuto")
pokemon.loc[140] = kabuto_data
kabutops_data = scrap_pokemon_details("Kabutops")
pokemon.loc[141] = kabutops_data

In [239]:
# Aerodactyl line and snorlax line
aerodactyl_data = scrap_pokemon_details("Aerodactyl")
aerodactyl_data["ability"] = "Rock Head / Pressure"
aerodactyl_data["hidden_ability"] = "Unnerve"
pokemon.loc[142] = aerodactyl_data
snorlax_data = scrap_pokemon_details("Snorlax")
pokemon.loc[143] = snorlax_data

In [240]:
# Legendary birds
articuno_data = scrap_pokemon_details("Articuno")
articuno_data["ability"] = "Pressure"
articuno_data["hidden_ability"] = "Snow Cloak"
articuno_data["category"] = "Freeze Pokemon" 
pokemon.loc[144] = articuno_data
zapdos_data = scrap_pokemon_details("Zapdos")
zapdos_data["ability"] = "Pressure"
zapdos_data["hidden_ability"] = "Static"
zapdos_data["category"] = "Electric Pokemon" 
pokemon.loc[145] = zapdos_data
moltres_data = scrap_pokemon_details("Moltres")
moltres_data["ability"] = "Pressure"
moltres_data["hidden_ability"] = "Flame Body"
moltres_data["category"] = "Flame Pokemon" 
pokemon.loc[146] = moltres_data

In [248]:
# Dratini line 
dratini_data = scrap_pokemon_details("Dratini")
pokemon.loc[147] = dratini_data
dragonair_data = scrap_pokemon_details("Dragonair")
pokemon.loc[148] = dragonair_data
dragonite_data = scrap_pokemon_details("Dragonite")
pokemon.loc[149] = dragonite_data

In [250]:
# Mew and mewtwo 
mewtwo_data = scrap_pokemon_details("Mewtwo")
mewtwo_data["ability"] = "Pressure"
mewtwo_data["hidden_ability"] = "Unnerve"
pokemon.loc[150] = mewtwo_data
mew_data = scrap_pokemon_details("Mew")
pokemon.loc[151] = mew_data

In [254]:
# Chikorita line 
chikorita = scrap_pokemon_details("Chikorita")
pokemon.loc[152] = chikorita
bayleef = scrap_pokemon_details("Bayleef")
pokemon.loc[153] = bayleef
meganium = scrap_pokemon_details("Meganium")
pokemon.loc[154] = meganium

In [257]:
# Cyndaquil line
cyndaquil = scrap_pokemon_details("Cyndaquil")
pokemon.loc[155] = cyndaquil
quilava = scrap_pokemon_details("Quilava")
pokemon.loc[156] = quilava
typhlosion = scrap_pokemon_details("Typhlosion")
typhlosion["ability"] = "Blaze"
typhlosion["hidden_ability"] = "Flash Fire"
typhlosion["category"] = "Volcano Pokemon"
pokemon.loc[157] = typhlosion

In [259]:
# totodile line 
totodile = scrap_pokemon_details("Totodile")
pokemon.loc[158] = totodile
croconaw = scrap_pokemon_details("Croconaw")
pokemon.loc[159] = croconaw
feraligatr = scrap_pokemon_details("Feraligatr")
pokemon.loc[160] = feraligatr

In [261]:
# sentret line 
sentret = scrap_pokemon_details("Sentret")
pokemon.loc[161] = sentret
furret = scrap_pokemon_details("Furret")
pokemon.loc[162] = furret

In [263]:
# hoothoot line 
hoothoot = scrap_pokemon_details("Hoothoot")
pokemon.loc[163] = hoothoot
noctowl = scrap_pokemon_details("Noctowl")
pokemon.loc[164] = noctowl

In [265]:
# ledyba and spinarak line 
ledyba = scrap_pokemon_details("Ledyba")
pokemon.loc[165] = ledyba
ledian = scrap_pokemon_details("Ledian")
pokemon.loc[166] = ledian
spinarak = scrap_pokemon_details("Spinarak")
pokemon.loc[167] = spinarak
ariados = scrap_pokemon_details("Ariados")
pokemon.loc[168] = ariados

In [267]:
# Crobat and chichou line 
crobat = scrap_pokemon_details("Crobat")
pokemon.loc[169] = crobat
chinchou = scrap_pokemon_details("Chinchou")
pokemon.loc[170] = chinchou
lanturn = scrap_pokemon_details("Lanturn")
pokemon.loc[171] = lanturn

In [269]:
# baby pokemon
pichu = scrap_pokemon_details("Pichu")
pokemon.loc[172] = pichu
cleffa = scrap_pokemon_details("Cleffa")
pokemon.loc[173] = cleffa
igglybuff = scrap_pokemon_details("Igglybuff")
pokemon.loc[174] = igglybuff
togepi = scrap_pokemon_details("Togepi")
pokemon.loc[175] = togepi 
togetic = scrap_pokemon_details("Togetic")
pokemon.loc[176] = togetic

In [271]:
# natu line 
natu = scrap_pokemon_details("Natu")
pokemon.loc[177] = natu
xatu = scrap_pokemon_details("Xatu")
pokemon.loc[178] = xatu

In [275]:
# mareep line
mareep = scrap_pokemon_details("Mareep")
pokemon.loc[179] = mareep
flaaffy = scrap_pokemon_details("Flaaffy")
pokemon.loc[180] = flaaffy
ampharos = scrap_pokemon_details("Ampharos")
ampharos["ability"] = "Static"
ampharos["hidden_ability"] = "Plus"
pokemon.loc[181] = ampharos

In [277]:
# bellossom and maril
bellossom = scrap_pokemon_details("Bellossom")
pokemon.loc[182] = bellossom
marill = scrap_pokemon_details("Marill")
pokemon.loc[183] = marill
azumarill = scrap_pokemon_details("Azumarill")
pokemon.loc[184] = azumarill
sudowoodo = scrap_pokemon_details("Sudowoodo")
pokemon.loc[185] = sudowoodo
politoed = scrap_pokemon_details("Politoed")
pokemon.loc[186] = politoed

In [12]:
# hopip line
hoppip = scrap_pokemon_details("Hoppip")
pokemon.loc[187] = hoppip
skiploom = scrap_pokemon_details("Skiploom")
pokemon.loc[188] = skiploom
jumpluff = scrap_pokemon_details("Jumpluff")
pokemon.loc[189] = jumpluff

In [14]:
# Aipom, sunkern
aipom = scrap_pokemon_details("Aipom")
pokemon.loc[190] = aipom
sunkern = scrap_pokemon_details("Sunkern")
pokemon.loc[191] = sunkern
sunflora = scrap_pokemon_details("Sunflora")
pokemon.loc[192] = sunflora
yanma = scrap_pokemon_details("Yanma")
pokemon.loc[193] = yanma

In [16]:
# wooper
wooper = scrap_pokemon_details("Wooper")
wooper["category"] = "Water Fish Pokemon"
wooper["ability"] = "Damp / Water Absorb"
wooper["hidden_ability"] = "Unaware"
pokemon.loc[194] = wooper
quagsire = scrap_pokemon_details("Quagsire")
pokemon.loc[195] = quagsire

In [18]:
# Eevvelutions
espeon = scrap_pokemon_details("Espeon")
pokemon.loc[196] = espeon
umbreon = scrap_pokemon_details("Umbreon")
pokemon.loc[197] = umbreon
murkrow = scrap_pokemon_details("Murkrow")
pokemon.loc[198] = murkrow

In [20]:
# Slowking
slowking = scrap_pokemon_details("Slowking")
slowking["ability"] = "Oblivious / Own Tempo"
slowking["hidden_ability"] = "Regenerator"
slowking["category"] = "Royal Pokemon"
pokemon.loc[199] = slowking
misdreavus = scrap_pokemon_details("Misdreavus")
pokemon.loc[200] = misdreavus
unknown = scrap_pokemon_details("Unknown")
pokemon.loc[201] = unknown
wobbuffet = scrap_pokemon_details("Wobbuffet")
pokemon.loc[202] = wobbuffet
girafarig = scrap_pokemon_details("Girafarig")
pokemon.loc[203] = girafarig

In [22]:
# Pineco line
pineco = scrap_pokemon_details("Pineco")
pokemon.loc[204] = pineco
forretress = scrap_pokemon_details("Forretress")
pokemon.loc[205] = forretress
pokemon.loc[206] = scrap_pokemon_details("Dunsparce")

In [24]:
# gligar, sleelix
pokemon.loc[207] = scrap_pokemon_details("Gligar")
steelix = scrap_pokemon_details("Steelix")
steelix["ability"] = "Rock Head / Sturdy"
steelix["hidden_ability"] = "Sheer Force"
pokemon.loc[208] = steelix
pokemon.loc[209] = scrap_pokemon_details("Snubbull")
pokemon.loc[210] = scrap_pokemon_details("Granbull")

In [27]:
# Qilfish and scizor
pokemon.loc[211] = scrap_pokemon_details("Qwilfish")
scizor = scrap_pokemon_details("Scizor")
scizor["ability"] = "Swarm / Technician"
scizor["hidden_ability"] = "Light Metal"
pokemon.loc[212] = scizor
pokemon.loc[213] = scrap_pokemon_details("Shuckle")

In [33]:
# Herracross
heracross = scrap_pokemon_details("Heracross")
heracross["ability"] = "Swarm / Guts"
heracross["hidden_ability"] = "Moxie"
pokemon.loc[214] = heracross
pokemon.loc[215] = scrap_pokemon_details("Sneasel")
pokemon.loc[216] = scrap_pokemon_details("Teddiursa")
pokemon.loc[217] = scrap_pokemon_details("Ursaring")
pokemon.loc[218] = scrap_pokemon_details("Slugma")
pokemon.loc[219] = scrap_pokemon_details("Magcargo")
pokemon.loc[220] = scrap_pokemon_details("Swinub")
pokemon.loc[221] = scrap_pokemon_details("Piloswine")

In [35]:
# corsola - houndoom
corsola = scrap_pokemon_details("Corsola")
corsola["ability"] = "Hustle / Natural Cure"
corsola["hidden_ability"] = "Regenerator"
pokemon.loc[222] = corsola
pokemon.loc[223] = scrap_pokemon_details("Remoraid")
pokemon.loc[224] = scrap_pokemon_details("Octillery")
pokemon.loc[225] = scrap_pokemon_details("Delibird")
pokemon.loc[226] = scrap_pokemon_details("Mantine")
pokemon.loc[227] = scrap_pokemon_details("Skarmory")
pokemon.loc[228] = scrap_pokemon_details("Houndour")
houndoom = scrap_pokemon_details("Houndoom")
houndoom["ability"] = "Early Bird / Flash Fire"
houndoom["hidden_ability"] = "Unnerve"
pokemon.loc[229] = houndoom

In [38]:
# kingdra
pokemon.loc[230] = scrap_pokemon_details("Kingdra")
pokemon.loc[231] = scrap_pokemon_details("Phanpy")
pokemon.loc[232] = scrap_pokemon_details("Donphan")
pokemon.loc[233] = scrap_pokemon_details("Porygon2")
pokemon.loc[234] = scrap_pokemon_details("Stantler")
pokemon.loc[235] = scrap_pokemon_details("Smeargle")
pokemon.loc[236] = scrap_pokemon_details("Tyrogue")
pokemon.loc[237] = scrap_pokemon_details("Hitmontop")
pokemon.loc[238] = scrap_pokemon_details("Smoochum")
pokemon.loc[239] = scrap_pokemon_details("Elekid")
pokemon.loc[240] = scrap_pokemon_details("Magby")

In [40]:
# Milktank
pokemon.loc[241] = scrap_pokemon_details("Miltank")
pokemon.loc[242] = scrap_pokemon_details("Blissey")

In [44]:
#Legendary beasts
raikou = scrap_pokemon_details("Raikou")
raikou["ability"] = "Pressure"
raikou["hidden_ability"] = "Inner Focus"
pokemon.loc[243] = raikou
entei = scrap_pokemon_details("Entei")
entei["ability"] = "Pressure"
entei["hidden_ability"] = "Inner Focus"
pokemon.loc[244] = entei
suicune = scrap_pokemon_details("Suicune")
suicune["ability"] = "Pressure"
suicune["hidden_ability"] = "Inner Focus"
pokemon.loc[245] = suicune

In [46]:
# larvitar line 
pokemon.loc[246] = scrap_pokemon_details("Larvitar")
pokemon.loc[247] = scrap_pokemon_details("Pupitar")
tyranitar = scrap_pokemon_details("Tyranitar")
tyranitar["ability"] = "Sand Stream"
tyranitar["hidden_ability"] = "Unnerve"
pokemon.loc[248] = tyranitar
pokemon.loc[249] = scrap_pokemon_details("Lugia")
pokemon.loc[250] = scrap_pokemon_details("Ho-Oh")
pokemon.loc[251] = scrap_pokemon_details("Celebi")

In [49]:
# treeko line 
pokemon.loc[252] = scrap_pokemon_details("Treecko")
pokemon.loc[253] = scrap_pokemon_details("Grovyle")
sceptile = scrap_pokemon_details("Sceptile")
sceptile["ability"] = "Overgrow"
sceptile["hidden_ability"] = "Unburden"
pokemon.loc[254] = sceptile

In [ ]:
# torchic line
pokemon.loc[255] = scrap_pokemon_details("Torchic")

In [51]:
pokemon.to_csv("/home/user/Desktop/Data_analysis_projects/Pokemon/data.csv")
pokemon[251:270]

,number,name,type,ability,hidden_ability,gender_ratio,hatch_cycle,egg_group,Base_Experience_Yield,height,weight,leveling_rate,color,base_friendship,category
252,#0252,Treecko,Grass,Overgrow,Unburden,"87.5% male, 12.5% female",20cycles,Monster and Dragon,62,0.5 m,5.0 kg,Medium Slow,Green,70,Wood Gecko Pokémon
253,#0253,Grovyle,Grass,Overgrow,Unburden,"87.5% male, 12.5% female",20cycles,Monster and Dragon,142,0.9 m,21.6 kg,Medium Slow,Green,70,Wood Gecko Pokémon
254,#0254,Sceptile,Grass,Overgrow,Unburden,"87.5% male, 12.5% female",20cycles,Monster and Dragon,239,1.7 m,52.2 kg,Medium Slow,Green,70,Forest Pokémon
255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
257,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
259,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
260,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
261,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [122]:
text = requests.get("https://bulbapedia.bulbagarden.net/wiki/Wigglytuff_(Pok%C3%A9mon)").text
soup = BeautifulSoup(text, "lxml")
data = soup.find_all("td", class_ = "roundy")

In [126]:
# ability and hidden ability
ability_data = data[2]
ability_data2 = ability_data.find_all(["a", "span"])
clear_ability = []
junk_text = ["Cacophony", "Abilities", "Gen IV+", "Battle Bond", "Battle Bond Greninja", "Gen VI+"]

for i in ability_data2:
    text = i.text.strip()
    if text and text not in junk_text:
        clear_ability.append(text)
        
if len(clear_ability) > 5:
    ability = clear_ability[0], clear_ability[2]
    hidden_ability = clear_ability[4]
elif len(clear_ability) < 5:
    ability = clear_ability[0]
    hidden_ability = clear_ability[2]
print(len(clear_ability), ability, hidden_ability)
clear_ability

6 ('Cute Charm', 'Competitive') Frisk


['Cute Charm', 'Cute Charm', 'Competitive', 'Competitive', 'Frisk', 'Frisk']